# Script to Create Figure 4 - Seasonal Means Trends for each EBUS and Subregion

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
from matplotlib.pyplot import scatter
import math
import glob
import warnings 
warnings.simplefilter('ignore') 

In [ ]:
#open file
trends = pd.read_csv('../../data/seasonal_trends_percent_Modified-Subregions_new.csv')

#create a column for significant trends
trends['sig'] = ''
#if the monte carlo p value is less than 0.025 (this is one tailed test)
trends.loc[trends['mc.p.value'] <= 0.025, 'sig'] = '*'

#add a column for subregion numbers
trends['Index_num'] = np.nan
trends.loc[(trends['Subregion'] == 'Poleward') & (trends['Season'] == 'Winter'), 'Index_num'] = 0
trends.loc[(trends['Subregion'] == 'Poleward') & (trends['Season'] == 'Spring'), 'Index_num'] = 1
trends.loc[(trends['Subregion'] == 'Poleward') & (trends['Season'] == 'Summer'), 'Index_num'] = 2
trends.loc[(trends['Subregion'] == 'Poleward') & (trends['Season'] == 'Fall'), 'Index_num'] = 3
trends.loc[(trends['Subregion'] == 'Central') & (trends['Season'] == 'Winter'), 'Index_num'] = 6 
trends.loc[(trends['Subregion'] == 'Central') & (trends['Season'] == 'Spring'), 'Index_num'] = 7
trends.loc[(trends['Subregion'] == 'Central') & (trends['Season'] == 'Summer'), 'Index_num'] = 8
trends.loc[(trends['Subregion'] == 'Central') & (trends['Season'] == 'Fall'), 'Index_num'] = 9
trends.loc[(trends['Subregion'] == 'Equatorward') & (trends['Season'] == 'Winter'), 'Index_num'] = 12
trends.loc[(trends['Subregion'] == 'Equatorward') & (trends['Season'] == 'Spring'), 'Index_num'] = 13
trends.loc[(trends['Subregion'] == 'Equatorward') & (trends['Season'] == 'Summer'), 'Index_num'] = 14
trends.loc[(trends['Subregion'] == 'Equatorward') & (trends['Season'] == 'Fall'), 'Index_num'] = 15

# trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Poleward') & (trends['Season'] == 'Winter'), 'Index_num'] = 2
# trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Poleward') & (trends['Season'] == 'Spring'), 'Index_num'] = 3
# trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Poleward') & (trends['Season'] == 'Summer'), 'Index_num'] = 4
# trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Poleward') & (trends['Season'] == 'Fall'), 'Index_num'] = 5

# trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Equatorward') & (trends['Season'] == 'Winter'), 'Index_num'] = 10
# trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Equatorward') & (trends['Season'] == 'Spring'), 'Index_num'] = 11
# trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Equatorward') & (trends['Season'] == 'Summer'), 'Index_num'] = 12
# trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Equatorward') & (trends['Season'] == 'Fall'), 'Index_num'] = 13


#add column for colors 
trends['Color'] = 'NA'
trends.loc[trends['Season'] == 'Winter', 'Color'] = '#377eb8'
trends.loc[trends['Season'] == 'Spring', 'Color'] = '#984ea3'
trends.loc[trends['Season'] == 'Summer', 'Color'] = '#4daf4a'
trends.loc[trends['Season'] == 'Fall', 'Color'] = '#e41a1c'

#replace the - with / in Iberian/Canary
trends['EBUS'] = trends['EBUS'].str.replace('-','/')
trends

In [ ]:
#create a 3x4 figure 
fig, axs = plt.subplots(2, 2, figsize=(16, 16), dpi = 300)

#loop through all of the columns/ebus
for ax, figure_label, ebus in zip(axs.ravel(), ['A', 'B', 'C', 'D'], ['California', 'Iberian/Canary', 'Humboldt', 'Benguela']):
    
    #filter the data
    temp = trends[(trends['EBUS'] == ebus)]

    #calculate the confidence intervals
    ytop = temp['upper.CL']-temp['Year.trend']
    ybot = temp['Year.trend']-temp['lower.CL']

    #plot the data
    ax.scatter(temp['Index_num'], temp['Year.trend'], c = temp['Color'], edgecolor = 'k', s = 200, zorder = 2, linewidth = 2)
    #add the error bars
    ax.errorbar(temp['Index_num'], temp['Year.trend'], yerr =(ybot, ytop), fmt='.k', 
                    c = 'k', capsize=8, zorder = 1, linewidth = 2, markeredgewidth=2)

    #add a grid to the data
    #ax.grid(alpha=0.5, zorder=0)

    #remove the axis titles for individual plots
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    #set tick labels
    ax.tick_params('both', labelsize=14, width = 2, length = 6)
    #increase line width of plot border
    [x.set_linewidth(2) for x in ax.spines.values()]

    #add space around the x-axis and add three subregion labels
    ax.set_xticks([-0.75, 1.5, 7.5, 13.5, 15.75])
    ax.set_xticklabels(['', 'Poleward', 'Central', 'Equatorward', ''], fontsize = 16)

    #add a line for 0
    ax.axhline(c = 'k', linewidth = 2, alpha=0.5, linestyle = 'dashed', zorder = 0)

    #add a title for each EBUS
    ax.set_title('(' + figure_label + ') ' + ebus, fontsize = 24, loc = 'left', pad = 10)

    #set the y-axis values so all the ebus are the same
    ax.set_yticks([-2.5, -2.0, -1.0, 0, 1.0, 2.0, 2.25])
    ax.set_yticklabels(['', '-2.0', '-1.0', '0', '1.0', '2.0', ''])

    #add the markers for the significance 
    for index, txt in enumerate(temp['sig']):
        ax.annotate(txt, (temp['Index_num'].iloc[index]-0.25, 1.9), fontsize = 32, weight='bold')
        

#adjust the figure
fig.tight_layout() 
fig.subplots_adjust(top = 0.9, bottom=0.01, hspace=0.2, wspace=0.1)

#add text elements for the axis labels
fig.text(-0.018, 0.43, 'Trend (percent area / year)', c='k', fontsize = 22, rotation = 90)

#add a legend for the whole figure
colors = ['#377eb8', '#984ea3', '#4daf4a', '#e41a1c']
points = [scatter([0], [0], color=c, edgecolor = 'k', s = 0) for c in colors]
labels = ['Winter', 'Spring', 'Summer', 'Fall']
lgnd = plt.legend(points, labels, loc='center left', bbox_to_anchor=(1, 1.1), prop={'size': 20}, handletextpad=0.1)
for handle in lgnd.legendHandles:
    handle.set_sizes([130])
    
#rescale axes for the last plot (adding legend causes issues)
ax.set_xticks([-0.75, 1.5, 7.5, 13.5, 15.75])
ax.set_xticklabels(['', 'Poleward', 'Central', 'Equatorward', ''], fontsize = 16)

ax.set_yticks([-2.5, -2.0, -1.0, 0, 1.0, 2.0, 2.25])
ax.set_yticklabels(['', '-2.0', '-1.0', '0', '1.0', '2.0', ''])

#save figure
plt.savefig('../../figures/Figure4_Seasonal_Trends.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()